In [1]:
import os, pickle, json
import numpy as np
from glob import glob
from tqdm import tqdm
from pyannote.core import notebook, Segment
from intervaltree import Interval, IntervalTree
from config import config
from collections import defaultdict

In [2]:
asr = glob(os.path.join(config['whisper']['output_path'], "*"))
asr.sort()
asr = {os.path.basename(x):x for x in asr}

In [3]:
diar = glob(os.path.join(config['pyannote']['diar_output_path'], "*"))
diar.sort()
diar = {os.path.basename(x):x for x in diar}

In [4]:
all_data = {}
for pod, pod_path in tqdm(asr.items()):
    
    with open(pod_path, 'rb') as f:
        segments = pickle.load(f)
    text = ''
    start = None
    
    all_data[pod] = {}
    counter = 1 
    for j, k in enumerate(range(len(segments))):


        seg = segments[k]
        s_start, end = seg.start, seg.end
        if start == None:
            start = s_start

        if (k+1) == len(segments):
            end = min(end, end+0.5) #change to end+1, end_audio
            all_data[pod][counter] = {
                'start':start,
                'end':end,
                'text':text.strip(),
            }
            continue

        next_seg_start = segments[k+1].start
        if (next_seg_start - end) > 0.2:
            text += seg.text
            end = min((next_seg_start - 0.05), end+0.5)
            all_data[pod][counter] = {
                'start':start,
                'end':end,
                'text':text.strip(),
            }
            start = None
            text = ''
            counter += 1
        else:
            text += seg.text

100%|███████████████████████████████████████| 6790/6790 [02:53<00:00, 39.10it/s]


In [5]:
for pod in tqdm(all_data.keys()):
    if pod not in diar:
        continue
    with open(diar[pod], 'rb') as f:
        diarization = pickle.load(f)
        
    d = [list(x) for x in list(diarization.itertracks(yield_label=True))]
    d = IntervalTree(
        Interval(info.start, info.end, "%s-%s" % (speaker, buf)) for info, buf, speaker in d
    )


    for seg in all_data[pod].keys():
        start, end = all_data[pod][seg]['start'], all_data[pod][seg]['end']
        speaking_utt = sorted(list(d.overlap(start, end)))
        speakers = list(set([x[2] for x in speaking_utt]))
        all_data[pod][seg]['speakers'] = speakers
        all_data[pod][seg]['num_speakers'] = len(speakers)


100%|███████████████████████████████████████| 6790/6790 [01:43<00:00, 65.46it/s]


In [6]:
with open(config['json_path'], 'w') as fp:
    json.dump(all_data, fp)